# Evaluation of Q&A Bot Responses

This notebook evaluates the quality of bot answers using various metrics:
- BERTScore (using DeepPavlov/rubert-base-cased model)
- ROUGE-1 and ROUGE-2 scores

We'll analyze the answers from the bot against reference answers to assess quality and accuracy.

In [8]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## Setup Path and Import Evaluator

First, we'll set up the Python path to include our source directory and import the necessary modules.

In [9]:
def setup_path() -> None:
    """Add src directory to Python path."""
    src_path = str(Path.cwd().parent / 'src')
    
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"Added {src_path} to Python path")
    
    try:
        import metrics
        print("Successfully imported metrics module")
    except ImportError as e:
        print(f"Error importing metrics module: {e}")
        print(f"Current sys.path: {sys.path}")
        raise

setup_path()
from metrics.evaluator import Evaluator

Successfully imported metrics module


## Load Q&A Data

Here we define our Q&A pairs for evaluation.

In [10]:
qa_data = [
            {
                'question': '3.1 назови состав изделия пак угледоры',
                'bot_answer': '''Состав изделия ПАК АИС Углеводороды:
– ПАК АИС в сборе (1 компл.);
– комплект запасных частей, инструментов и принадлежностей (1 компл.). (Недостаточно информации.)''',
                'reference_answer': '''3.1 Состав изделия
Состав изделия ПАК АИС Углеводороды:
– ПАК АИС в сборе (1 компл.);
– комплект запасных частей, инструментов и принадлежностей (1 компл.);
– общее программное обеспечение (1 компл.);
– специальное программное обеспечение (1 компл.);
– комплект лицензий на всё применяемое программное обеспечение (1 компл.);
– монтажный комплект (при необходимости).'''
            },
            {
                'question': '3.1.3. Что должно происходить на уровне АРМ и что должно быть обеспечено?',
                'bot_answer': 'На уровне АРМ операторов должна происходить визуализация технологических параметров. Должно быть обеспечено отсутствие возможности влияния на метрологически значимое ПО на уровне АРМ.',
                'reference_answer': 'На уровне АРМ операторов должна происходить визуализация технологических параметров. Должно быть обеспечено отсутствие возможности влияния на метрологически значимое ПО на уровне АРМ.'
            },
            {
                'question': 'Выведи Потребляемую мощность и средний срок службы из таблицы 3.2.2 характеристики Пак углеводороды',
                'bot_answer': 'Недостаточно информации. В контексте не содержится информации о таблице 3.2.2, а также о характеристиках Пак углеводороды, кроме тех, которые приведены в таблице 1. В таблице 1 не содержится информации о Потребляемой мощности и среднем сроке службы.',
                'reference_answer': 'Потребляемую мощность - не более 2квт\nсредний срок службы - 10'
            },
            {
                'question': 'Перечисли все наименования выполняемых работ и их срок выполнения из таблицы 12 выполнение окр',
                'bot_answer': 'Недостаточно информации.',
                'reference_answer': 'Недоастаточно информации.'
            },
            {
                'question': 'Выведи отчетную документацию для 1.2 Сбор информации о технологическом процессе пилотного объекта: данные технологического регламента, технологические схемы, чертежи, архивные данные АСУ ТП и аналитического и инструментального контроля из таблицы 12 выполнение окр',
                'bot_answer': '''- описание результатов обследования пилотного объекта и возможных ограничений (при наличии) для проведения моделирования;
- анализ технологических процессов производственных объектов отрасли, распространенности производств, возможности проведения моделирования;
-перечень данных, необходимых для проведения моделирования;
- требования к структуре и функционированию ПАК АИС Углеводороды;
- основные технические решения по разработке ПАК АИС Углеводороды;
- техническое задание на создание ПАК АИС Углеводороды.''',
                'reference_answer': '''Научно-технический отчет по ОКР, включающий:
    - описание результатов обследования пилотного объекта и возможных ограничений (при наличии) для проведения моделирования;
    - анализ технологических процессов производственных объектов отрасли, распространенности производств, возможности проведения моделирования;
    -перечень данных, необходимых для проведения моделирования;
    - требования к структуре и функционированию ПАК АИС Углеводороды;
    - основные технические решения по разработке ПАК АИС Углеводороды;
    - техническое задание на создание ПАК АИС Углеводороды.'''
            }
        ]

# Convert to DataFrame for easier handling
df = pd.DataFrame(qa_data)
df.head()

,question,bot_answer,reference_answer
0,3.1 назови состав изделия пак угледоры,Состав изделия ПАК АИС Углеводороды:\n– ПАК АИ...,3.1 Состав изделия\nСостав изделия ПАК АИС Угл...
1,3.1.3. Что должно происходить на уровне АРМ и ...,На уровне АРМ операторов должна происходить ви...,На уровне АРМ операторов должна происходить ви...
2,Выведи Потребляемую мощность и средний срок сл...,Недостаточно информации. В контексте не содерж...,Потребляемую мощность - не более 2квт\nсредний...
3,Перечисли все наименования выполняемых работ и...,Недостаточно информации.,Недоастаточно информации.
4,Выведи отчетную документацию для 1.2 Сбор инфо...,- описание результатов обследования пилотного ...,"Научно-технический отчет по ОКР, включающий:\n..."


## Evaluate Q&A Pairs

Now we'll evaluate each Q&A pair using our metrics.

In [11]:
# Initialize evaluator
evaluator = Evaluator(bert_model_name='DeepPavlov/rubert-base-cased')

# Calculate metrics for each pair
metrics_results = []

for idx, row in df.iterrows():
    print(f"Processing pair {idx + 1}/{len(df)}")
    if row['bot_answer'] and row['reference_answer']:
        try:
            scores = evaluator.evaluate_text_similarity(
                row['bot_answer'],
                row['reference_answer']
            )
            print(f"Scores for pair {idx}: {scores}")
            
            # Extract metrics safely
            bert_score = scores.get('bert_score', 0.0)
            rouge_scores = scores.get('rouge', {})
            rouge1 = rouge_scores.get('rouge1', 0.0)
            rouge2 = rouge_scores.get('rouge2', 0.0)
            
            metrics_results.append({
                'question': row['question'],
                'bert_score': bert_score,
                'rouge1': rouge1,
                'rouge2': rouge2
            })
        except Exception as e:
            print(f"Error processing pair {idx}: {e}")
            print(f"Scores object: {scores}")
            continue

metrics_df = pd.DataFrame(metrics_results)
metrics_df = metrics_df.rename(columns={
    'bert_score': 'BERTScore',
    'rouge1': 'ROUGE-1',
    'rouge2': 'ROUGE-2'
})

metrics_df

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processing pair 1/5
Scores for pair 0: {'bert_score': 0.8622425198554993, 'rouge': {'rouge-1': {'precision': 0.9142857142857143, 'recall': 0.42105263157894735, 'f1': 0.5765765765765766}, 'rouge-2': {'precision': 0.8235294117647058, 'recall': 0.37333333333333335, 'f1': 0.5137614678899083}}}
Processing pair 2/5
Scores for pair 1: {'bert_score': 0.9999999403953552, 'rouge': {'rouge-1': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'rouge-2': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}}
Processing pair 3/5
Scores for pair 2: {'bert_score': 0.7205520272254944, 'rouge': {'rouge-1': {'precision': 0.04878048780487805, 'recall': 0.18181818181818182, 'f1': 0.07692307692307693}, 'rouge-2': {'precision': 0.0, 'recall': 0.0, 'f1': 0}}}
Processing pair 4/5
Scores for pair 3: {'bert_score': 0.9944342374801636, 'rouge': {'rouge-1': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1': 0.6666666666666666}, 'rouge-2': {'precision': 0.5, 'recall': 0.5, 'f1': 0.5}}}
Processing pair 5/5
S

,question,BERTScore,ROUGE-1,ROUGE-2
0,3.1 назови состав изделия пак угледоры,0.862243,0.0,0.0
1,3.1.3. Что должно происходить на уровне АРМ и ...,1.000000,0.0,0.0
2,Выведи Потребляемую мощность и средний срок сл...,0.720552,0.0,0.0
3,Перечисли все наименования выполняемых работ и...,0.994434,0.0,0.0
4,Выведи отчетную документацию для 1.2 Сбор инфо...,0.979120,0.0,0.0


## Analysis of Results

Let's look at the summary statistics and best/worst performing answers.

In [12]:
required_columns = ['BERTScore', 'ROUGE-1', 'ROUGE-2']
print("Summary Statistics:")
print(metrics_df[required_columns].describe())

if 'BERTScore' in metrics_df.columns:
    print("\nBest answer by BERTScore:")
    best_idx = metrics_df['BERTScore'].idxmax()
    print(f"Question: {df.iloc[best_idx]['question']}")
    print(f"Bot Answer: {df.iloc[best_idx]['bot_answer']}")
    print(f"Reference: {df.iloc[best_idx]['reference_answer']}")
    print(f"BERTScore: {metrics_df.iloc[best_idx]['BERTScore']:.3f}")
    
    print("\nWorst answer by BERTScore:")
    worst_idx = metrics_df['BERTScore'].idxmin()
    print(f"Question: {df.iloc[worst_idx]['question']}")
    print(f"Bot Answer: {df.iloc[worst_idx]['bot_answer']}")
    print(f"Reference: {df.iloc[worst_idx]['reference_answer']}")
    print(f"BERTScore: {metrics_df.iloc[worst_idx]['BERTScore']:.3f}")

Summary Statistics:
       BERTScore  ROUGE-1  ROUGE-2
count   5.000000      5.0      5.0
mean    0.911270      0.0      0.0
std     0.120592      0.0      0.0
min     0.720552      0.0      0.0
25%     0.862243      0.0      0.0
50%     0.979120      0.0      0.0
75%     0.994434      0.0      0.0
max     1.000000      0.0      0.0

Best answer by BERTScore:
Question: 3.1.3. Что должно происходить на уровне АРМ и что должно быть обеспечено?
Bot Answer: На уровне АРМ операторов должна происходить визуализация технологических параметров. Должно быть обеспечено отсутствие возможности влияния на метрологически значимое ПО на уровне АРМ.
Reference: На уровне АРМ операторов должна происходить визуализация технологических параметров. Должно быть обеспечено отсутствие возможности влияния на метрологически значимое ПО на уровне АРМ.
BERTScore: 1.000

Worst answer by BERTScore:
Question: Выведи Потребляемую мощность и средний срок службы из таблицы 3.2.2 характеристики Пак углеводороды
Bot Answ